# Lending Club Case Study

Group Members: 

<b>1. Jane Saldanha</b> <br>
<b>2. Anant Kumar Pramanik</b>

## Project Brief (Problem Statement)

We analyze the historical loan data of a large online loan marketplace specializing in lending various kind of loans to urban customers. The objective of the analysis is to:

1. Identify the driving factors behind loan defaults
2. Identify and recommend safe loan candidates so the company doesnt incur business loss by NOT lending to those candidates.

## Analysis Approach

1. <b>Data Understanding</b> - 
Identify quality issues , Interpret the correct meaning of vairables (Columns)


2. <b>Data Cleaning and Manipulation </b> - 
Data (like strings and dates) are corrected to the right type to facilitate analysis , Empty (Null) value columns and variables are removed


3. <b>Data Analysis </b> - 

    a. Use <i>univariate and Segmented Univariate analysis </i>to identify atleast 5 drivers (Indicators) of defaults
    
    b. Create new <i>Derrived metrics</i> wherever needed (whether business driven, Type driven or Data driven) to get  new maningful business insights
    
    c.<i>Bivariate and Multivariate analysis</i> are done to understand combination of loan default and loan success drivers which makes business sense.
    
    Also <i>Plot appropriate graphs</i> with to present the results of the above types of analysis and to make it easy to draw clear insights.
    
    
4. <b>Presentation and Recommendations </b> - 
A summary of the full analysis is created in a presentation format , explaining the insgihts drawn towards the default drivers and busines loss drivers with clear actionable suggestions for improvement in default reduction and slection of better loan canditates.



## IMPORTING LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import toolz
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

## READING AND VIEWING THE DATA

In [ ]:
df = pd.read_csv('loan.csv')

In [ ]:
df.head()

## SHAPE OF THE DATAFRAME

In [ ]:
df.shape

## DATA TYPES OF THE COLUMN IN DATAFRAME

In [ ]:
df.dtypes

In [ ]:
## In order to view all of them we can convert them in to a dictionary
df.dtypes.to_dict()
# we can open the full output in a text editor using vscode

## [IMPUTATION]:  MISSING VALUES - DROPPING COLUMNS

In [ ]:
def check_percentage(df):
    """
    Function to calculate the percentage of null values
    Input: A column of the dataframe as input
    Output: the percentage in float value
    """
    status = df.isnull().sum() / len(df)
    return status
    

In [ ]:
# dataframe to store the column name and the percentage of null for visualization
null_df = pd.DataFrame(columns=['Name', 'Value'])
# counter to add rows in the dataframe
count = 0
for col in df.columns:
    value =  float(check_percentage(df[col])) * 100
    print("The percentage of null values in the column {} are : {}".format(col, value))
    null_df.loc[count] = [col, value]
    count = count + 1
    

In [ ]:
# Let's visualize this for better understanding.
null_fig = px.treemap(null_df, path=['Name'], values='Value')
null_fig.show()

As we can see from the tree map that there are many columns which are 100% null which have no use to us for further analysis. But we cannot get a clear analysis on the rest of the null value percentages so we will now proceed with a histogram

In [ ]:
fig = px.histogram(null_df, x="Value", range_x=[0, 100])
fig.show()

As we can see in the plots above that there are over :
1. 56 columns that have more than 90% null values

2. One column in the range of 30% to 49% null values

3. One column in the range of 50% to 69% null values

4. 53 columns in the range of 0 to 9% null values

### Removing all the columns that have more than 50% null values

In [ ]:
df = df[null_df[null_df['Value']< 50]['Name'].values]

In [ ]:
print("The shape of the dataframe now is {}".format(df.shape))

In [ ]:
df.columns

## [IMPUTATION]: MISSING VALUES  - FILLING ROWS

In [ ]:
# the columns that still have null values 
df.columns[df.isnull().any()].tolist()

Let's deal with the rest of the columns one by one

### EMP_TITLE

In [ ]:
df['emp_title'].unique()

Replacing nan with Unknown

In [ ]:
df['emp_title'] = df['emp_title'].replace(np.nan, "Unknown")

### EMP_LENGTH

In [ ]:
## Removing non number characters from emp_length column
df['emp_length'] = df['emp_length'].replace({'< 1 year':'0'},regex=True)
df['emp_length'] = df['emp_length'].replace({'10+ years':'10'})
df['emp_length']= df['emp_length'].replace({'year':''}, regex=True)
df['emp_length'] = df['emp_length'].replace({'s':''},regex=True)
df['emp_length'].fillna('0', inplace=True)

In [ ]:
df['emp_length'].unique()

### Desc

In [ ]:
df.shape

In [ ]:
df['desc'].isna().sum() / 39717

In [ ]:
df[df['desc'] == ' '].shape

Dropping this column since it does not have any significant need in the analysis.  A lot of the rows are nan and blank and it does not make sense to be replaced by the mode 

In [ ]:
df.drop(['desc'], axis=1, inplace=True)

### pub_rec_bankrupcies

In [ ]:
df['pub_rec_bankruptcies'].mode()

Replacing the nan values with the mode

In [ ]:
df['pub_rec_bankruptcies'] = df['pub_rec_bankruptcies'].replace(np.nan, 0)

In [ ]:
df['pub_rec_bankruptcies'] = df['pub_rec_bankruptcies'].astype('str')

### TITLE

The column **title** is the loan title provided by the borrower. Lets see the kind of values that are provided

Checking the percentage of each title to understand if it makes sense to substitute the na values with mode or drop them

In [ ]:
def percent(x):
    return (x/df.shape[0]) * 100
#print(toolz.valmap(percent, df['title'].value_counts().to_dict()))
# remove comments in the end

 we will drop the titles that have NA values because they cannot be necessarily be replaced by the mode. 

In [ ]:
df = df[df['title'].isna() == False]

In [ ]:
print("Shape of the dataframe is : {}".format(df.shape))

### REVOL UTIL

Let's look in to the column **revol_util**: Revolving utilization compares your credit debt to your total credit limit. To calculate it, divided your balance by your credit limit. example: you have 2000 usd on a credit card and a 5k limit on it. your util is 40% if you have a second credit card with 5k limit but no debt then your util is 20%


Revolving credit is an agreement that permits an account holder to borrow money repeatedly up to a set dollar limit while repaying a portion of current balance due in regular payments

In [ ]:
df[df['revol_util'].isna() == True].shape

As we can see we have 50 null values in revol util, we need to either replace them or drop them.

Cleaning the revolving utilization column by removing the percentage and turning it into float.


Filling the null values with mean of the column

In [ ]:
mean_revol_util = df[df['revol_util'].isna() == False]['revol_util'].apply(lambda x: float(x[:-1])).mean()
mean_revol_util = str(mean_revol_util) + "%"

In [ ]:
df['revol_util'].fillna(value=mean_revol_util, inplace=True)

### LAST_PYMNT_D

Consider the 3rd column  **last_pymnt_d** : it consists of the month and the date. Let's check the distribution of the last payment date.

In [ ]:
def viz(df, col_name):
    """
    Visualizing the frequency count for  a column
    """
    last_payment_viz = df[col_name].value_counts().to_frame()
    last_payment_viz.reset_index(inplace=True)
    return last_payment_viz
    #px.bar(last_payment_viz, x="index", y=col_name)

In [ ]:
df['last_pymnt_d'].isna().sum()

There are 71 rows with last payment date as null

In [ ]:
last_payment_viz = viz(df, 'last_pymnt_d')
px.bar(last_payment_viz, x="index", y='last_pymnt_d')

As we can see that majority of the last payment dates happen on May 16th.

We can replace the null values by the mode of the column

In [ ]:
df['last_pymnt_d'].fillna(df['last_pymnt_d'].mode()[0], inplace=True)

### LAST CREDIT PULL D

Consider the column **last_credit_pull_d** : The most recent month Lending Club pulled credit for this loan



In [ ]:
df['last_credit_pull_d'].isna().sum()

There are 2 null values in the column

In [ ]:
last_credit_viz = viz(df, 'last_credit_pull_d')
px.bar(last_credit_viz, x="index", y='last_credit_pull_d')

As we can see that majority of the last credit was pulled at May 16 so the we can now fill null values with mode

In [ ]:
df['last_credit_pull_d'].fillna(df['last_credit_pull_d'].mode()[0], inplace=True)

### COLLECTIONS_12_MTHS_EX_MED

Meaning: Number of collections in 12 months excluding medical collections

A debt that has “gone to collections” means that the debt (a loan, credit card, line of credit or even unpaid bills) has gone unpaid for a significant amount of time, and is typically more than 90 days overdue

In [ ]:
df['collections_12_mths_ex_med'].isna().sum()

There are 56 null values in this column.
 
Let's see the kind of values that are present

In [ ]:
df['collections_12_mths_ex_med'].unique()

There are two values 0 and nan we can replace the nan values with 0 

In [ ]:
df['collections_12_mths_ex_med'].fillna(0, inplace=True)

### CHARGEOFF_WITHIN_12_MTHS

Meaning: Number of charge-offs within 12 months

A charge-off is a debt, for example on a credit card, that is deemed unlikely to be collected by the creditor because the borrower has become substantially delinquent after a period of time. However, a charge-off does not mean a write-off of the debt entirely. Having a charge-off can mean serious repercussions on your credit history and future borrowing ability.

In [ ]:
print("There are {} null values.".format(df['chargeoff_within_12_mths'].isna().sum()))

In [ ]:
df['chargeoff_within_12_mths'].unique()

There are two types of values 0 and nan and we will replace nan with 0

In [ ]:
df['chargeoff_within_12_mths'].fillna(0, inplace=True)

### TAX_LIENS

Description: Number of tax liens

A tax lien is a legal claim against the property of an individual or business that fails to pay taxes owed to the government

In [ ]:
df['tax_liens'].unique()

Filling the null values with 0

In [ ]:
df['tax_liens'].fillna(0, inplace=True)

Checking for null values

In [ ]:
df.isnull().sum()

### Pub Rec
Number of derogatory public records

In [ ]:
df['pub_rec'].unique()

Converting these into str format so that they remain as categorical variable

In [ ]:
df['pub_rec'] = df['pub_rec'].astype(str)

There are no more null values in the dataframe

## CHECKING THE DATAFRAME FOR DUPLICATE ROWS

In [ ]:
duplicate = df[df.duplicated()]
print(duplicate.shape)

There are no duplicate rows

## [DATA CLEANING] : CHANGING THE DATA TYPES OF THE COLUMN AND TAILORING THE DATA AS PER NEED

In [ ]:
#Dropping member id and id as they have no significant contribution to the analysis
df2 = df.drop(['id', 'member_id'], axis=1)

Let's review every column one by one for data cleaning

### Loan Amount

 The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

In [ ]:
df2['loan_amnt'].dtype

No changes in this column

### Funded Amount

The total amount committed to that loan at that point in time.

In [ ]:
df2['funded_amnt'].dtype

###  Funded Amount Inv

The total amount committed by investors for that loan at that point in time.

In [ ]:
df2['funded_amnt_inv'].dtype

### Term

The number of payments on the loan. Values are in months and can be either 36 or 60.

In [ ]:
df2['term'].dtype

It is of type object let's see how this could be modified to useful information

Changes that can be done on this column is removal of the months and retrieving the months in integer format

In [ ]:
df2['term'] = df2['term'].apply(lambda x: str(x.strip()[:2]))

In [ ]:
df2['term'].dtype

### Int_rate

Interest Rate on the loan

In [ ]:
df2['int_rate'].dtype

We can change this column by removing the percentage from the end of the value and turn into float

In [ ]:
df2['int_rate'] = df2['int_rate'].apply(lambda x: float(x[:-1]))

### Installment

The monthly payment owed by the borrower if the loan originates.

In [ ]:
df2['installment'].dtype

No changes required

### Grade

LC assigned loan grade

In [ ]:
df2['grade'].unique()

These are the different grades present. For Exploratory Data Analysis, we do not need to replace the values

### Sub_grade

LC assigned loan subgrade

In [ ]:
df2['sub_grade'].unique()

For each grade there are various subgrades ranging from value 1 to 5

### Home_Ownership

In [ ]:
df2['home_ownership'].unique()

As we can see here the various kinds of home ownerships are rent , own , mortgage , other and none.
None should not be replaced because not everyone owns a home. No  change required in this column

### Annual Income

The self-reported annual income provided by the borrower during registration.

In [ ]:
df2['annual_inc'].dtype

No change required in this column

### verification_status 

In [ ]:
df2['verification_status'].unique()

There are 3 values for this column verified, source verified and not verified

### Issue Date


The month which the loan was funded

In [ ]:
df2['issue_d'].unique()

We can see that the values are a string with month and year given. So we can break down this column into two seperate columns

In [ ]:
df2['issue_month'] = df2['issue_d'].apply(lambda x: x.split("-")[0])

In [ ]:
df2['issue_year'] = df2['issue_d'].apply(lambda x: int("20" + x.split("-")[1]))

After splitting these columns we can drop the issue_d column

In [ ]:
df2 = df2.drop(['issue_d'], axis=1)

### loan_status 

Current status of the loan

In [ ]:
df2['loan_status'].unique()

No changes in the column

### Payment plans

Indicates if a payment plan has been put in place for the loan

In [ ]:
df2['pymnt_plan'].unique()

Just has the value n

### url

In [ ]:
df2['url']

We can drop this column because the url does not have any significant contribution for the analysis

In [ ]:
df2.drop(['url'], axis=1, inplace=True)

### Purpose

A category provided by the borrower for the loan request. 

In [ ]:
df['purpose'].unique()

No changes in column for exploratory data analysis

### Title

In [ ]:
df2['title'].unique()

In [ ]:
print("The number of different titles are {}".format(len(df2['title'].unique())))

### zip_code   

In [ ]:
df2['zip_code'].unique()

The xx value in the end is to mask the data so removing it for further analysis

In [ ]:
df2['zip_code'] = df2['zip_code'].apply(lambda x: int(x[:-2]))

### addr_state

The state provided by the borrower in the loan application

In [ ]:
df2['addr_state'].unique()

No changes in the addr state

### dti

A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.


A DTI of 43% is typically the highest ratio a borrower can have and still get qualified for a mortgage, but lenders generally seek ratios of no more than 36%

In [ ]:
df2['dti'].describe()

No change in the column


### delinq_2yrs

The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years

The delinquency rate refers to the percentage of loans that are past due. It indicates the quality of a lending company’s or a bank’s loan portfolio.

In [ ]:
df2['delinq_2yrs'].unique()

No changes in the column

### earliest_cr_line


The month the borrower's earliest reported credit line was opened

In [ ]:
def year_change(x):
    year_end = int(x.split("-")[1])
    if year_end > 20:
        return int(str(19) + str(year_end))
    else:
        return int(str(200) + str(year_end))

In [ ]:
df2['earliest_cr_line_month'] = df2['earliest_cr_line'].apply(lambda x: x.split("-")[0])
df2['earliest_cr_line_month_year'] = df2['earliest_cr_line'].apply(lambda x: year_change(x))

In [ ]:
df2.drop(['earliest_cr_line'], axis=1, inplace=True)

### inq_last_6mths

The number of inquiries in past 6 months (excluding auto and mortgage inquiries)

In [ ]:
df2['inq_last_6mths'].unique()

No changes in the columns

### open_acc

The number of open credit lines in the borrower's credit file.

In [ ]:
df2['open_acc'].unique()

No changes in this column

### pub_rec

Number of derogatory public records



In [ ]:
df2['pub_rec'].unique()

In [ ]:
df2['pub_rec']

No changes in pub_rec col


### revol_bal

Total credit revolving balance


In [ ]:
df2['revol_bal']

No changes in column


### revol_util

In [ ]:
df2['revol_util'] = df2['revol_util'].apply(lambda x: float(x[:-1]))

Removed the % of the column and turned it into float

### total_acc

The total number of credit lines currently in the borrower's credit file


In [ ]:
df2['total_acc'].unique()

No changes in the column

### initial_list_status

The initial listing status of the loan. Possible values are – W, F



In [ ]:
df2['initial_list_status'].unique()

Has only one value; no change as of now

### out_prncp

Remaining outstanding principal for total amount funded



In [ ]:
df2['out_prncp'].unique()

No change in column



### out_prncp_inv

Remaining outstanding principal for portion of total amount funded by investors


In [ ]:
df2['out_prncp_inv'].unique()

No change in column


### total_pymnt

Payments received to date for total amount funded



In [ ]:
df2['total_pymnt']


No changes in column

### total_pymnt_inv

Payments received to date for portion of total amount funded by investors


In [ ]:
df2['total_pymnt_inv']

No changes in column

### total_rec_prncp

Principal received to date


In [ ]:
df2['total_rec_prncp']

No changes in column

### total_rec_int

Interest received to date


In [ ]:
df2['total_rec_int']

No changes in column


### total_rec_late_fee

Late fees received to date


In [ ]:
df2['total_rec_late_fee']

No changes in column

### recoveries

post charge off gross recovery


In [ ]:
df2['recoveries']

No changes in column

### collection_recovery_fee

post charge off collection fee



In [ ]:
df2['collection_recovery_fee']

No changes in the column

### last_pymnt_d

Last month payment was received


In [ ]:
df2['last_pymnt_d'].unique()

In [ ]:
df2['last_payment_month'] = df2['last_pymnt_d'].apply(lambda x: x.split("-")[0])

Dropping the last payment d column 

In [ ]:
df2.drop(['last_pymnt_d'], axis=1, inplace=True)

### last_pymnt_amnt

Last total payment amount received


In [ ]:
df2['last_pymnt_amnt']

No change in column

### last_credit_pull_d

The most recent month LC pulled credit for this loan


In [ ]:
df2['last_credit_pull_d'] = df2['last_credit_pull_d'].apply(lambda x: x.split("-")[0])

### collections_12_mths_ex_med

In [ ]:
df2['collections_12_mths_ex_med'].unique()

Drop the column since it has one value 

### policy_code

"publicly available policy_code=1

new products not publicly available policy_code=2"


In [ ]:
df2['policy_code'].unique()

Drop column since one value only

### application_type

In [ ]:
df2["application_type"].unique()

drop the columns that has one value

### acc_now_delinq 

In [ ]:

df2['acc_now_delinq'].unique()

Drop the column

### chargeoff_within_12_mths 

In [ ]:
df2['chargeoff_within_12_mths'].unique()

drop the column since it has only one value 

### delinq_amnt 

In [ ]:
df2['delinq_amnt'].unique()

drop the column


### tax_liens

In [ ]:
df2['tax_liens'].unique()

Dropping the column

### As we observed few of the columns have only one value. 

Hence dropping those columns

In [ ]:
df2.columns[df2.nunique() <= 1]

Dropping the columns

In [ ]:
df2 = df2[df2.columns[df2.nunique() > 1]]

In [ ]:
df2.shape

## [DATA CLEANING]: OUTLIER REMOVAL

In [ ]:
#ig = go.Figure(data=[go.Histogram(x=df2['loan_amnt'])])
#fig.show()
fig = make_subplots(rows=7, cols=4,
subplot_titles=tuple(df2._get_numeric_data().columns))
row = 1
col = 1
for i in df2._get_numeric_data().columns:
    fig.add_trace(go.Histogram(x=df2[i]),
     row = row, col = col)
    if col % 4 == 0:
        row = row + 1
        col = 1
    else:
        col = col + 1
# Overlay both histograms
fig.update_layout(barmode='overlay', width=800,
    height=2000)
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

If we see that majority of the columns are skewed either to the left or the right.

We can use IQR to remove outliers

Plotting the box plots to check the outliers

In [ ]:
vars = df2._get_numeric_data().columns
fig = make_subplots(rows=7, cols=4)
row = 1
col = 1
for i, var in enumerate(vars):
    
    fig.add_trace(
        go.Box(y=df2[var],
        name=var),
        row=row, col=col
    )
    if col % 4 == 0:
        row = row + 1
        col = 1
    else:
        col = col + 1

fig.update_traces(jitter=.3)
fig.update_layout(
    autosize=False,
    width=800,
    height=2000)

The dots above are the outliers


In [ ]:
def drop_outliers(df):
    """
    Calculating the outliers and returning the indexes
    """
    quartile_one = df.quantile(0.25)
    quartile_three = df.quantile(0.75)
    iqr = quartile_three - quartile_one
    index = ~((df<(quartile_one-1.5*iqr)) | (df>(quartile_three+1.5*iqr)))
    return index
    

In [ ]:
li = list(range(0, df2.shape[0]))

# Create a DataFrame index object
# using pd.Index() function
idx = pd.Index(li)

# Set the above DataFrame index object as the index
# using set_index() function
df2 = df2.set_index(idx)

In [ ]:
def drop_check_outlier(fig, df2, col_name, row_value, col_value):
    """
    Filtering the dataframe via indexes and plotting the box plots.
    """
    x = drop_outliers(df2[col_name])
    df2 = df2[x]
    li = list(range(0, df2.shape[0]))
    idx = pd.Index(li)
    df2 = df2.set_index(idx)
    fig.add_trace(
                go.Box(y=df2[col],
                name=col),
                row=row_value, col=col_value
        )

    return fig, df2

In [ ]:
row_value = 1
col_value = 1
fig = make_subplots(rows=9, cols=4)
for col in df2._get_numeric_data().columns:
    fig, df2 = drop_check_outlier(fig, df2, col, row_value, col_value)
    fig.update_traces(jitter=.3)
    if col_value % 4 == 0:
                row_value = row_value + 1
                col_value = 1
    else:
                col_value = col_value + 1
fig.update_layout(
                autosize=False,
                width=1000,
                height=2000)

As we can see in boxplots, there are few columns which have just one value. 

Dropping those columns as they do not have signficance

In [ ]:
df2 = df2[df2.columns[df2.nunique() > 1]]

In [ ]:
print("Final Shape of the DataFrame {}".format(df2.shape))

In [ ]:
df2.columns

## UNIVARIATE ANALYSIS

To understand the distribution of the columns present

In [ ]:
df2.shape

In [ ]:
df2.head()

Let's check the columns present 

In [ ]:
df2.columns

### Loan Amount

Let's analyze the Loan amount that was applied for by the customers

In [ ]:
df2['loan_amnt'].describe()

In [ ]:
fig = px.histogram(df2, x="loan_amnt", title="Frequency of the Loan Amount")
fig.show()

As we can see in the bar plot and describe function output:
1. The max number of applicants have applied for 10k USD Loan
2. The second max number of applicants have applied for loan from 6k to 6.2k
3. The max loan applied for by the applicant is 28k USD

### Employee Length

Understanding the distribution of employee length in the dataset

In [ ]:
df2['emp_length'].unique()

In [ ]:
df_emp = df2['emp_length'].value_counts().to_frame()
df_emp = df_emp.reset_index()
df_emp.columns = ["emp_length", "Count"]

In [ ]:
fig = px.bar(df_emp, x="emp_length", y="Count", title="Frequency of employee length", color_discrete_sequence=['darkgreen'])
fig.show()

We can see that most of the loans were given to employee with more than 10 years experience 

The possiblity of giving people with less than 1 year experience loans is the factor of age, high earning job positions and ability to pay loan faster because of lack of family responsbilities relatively.

### Number of derogatory public records


In [ ]:
df2.columns

### TERM

Let's analyze the term column and understand the kind of terms the bank allows

In [ ]:
df_term = df2['term'].value_counts().to_frame()

In [ ]:
df_term = df_term.reset_index()
df_term.columns = ["term", "Count"]

In [ ]:
fig = px.pie(df_term,values = "Count", names="term", hole=.3, title = "Frequncy of the loan terms")
fig.show()

We can see that the bank gives 36 months term  74.6% more than 60 months term. 


### INT_RATE

Let's now analyze the **int_rate** column

In [ ]:
df2['int_rate'].describe()

In [ ]:
df2['int_rate'].mode()

In [ ]:
group_labels = ['int_rate'] # name of the dataset
fig = ff.create_distplot([df2['int_rate']], group_labels, colors=['green'])
fig.update_layout(width=800,
    height=1000, title="Interest Rate sanctioned by the bank")
fig.show()

Majority of the loans that the bank gives has a rate of interest of 7.5% and maximum the rate of interest given is 22.11%

### INSTALLEMENT

In [ ]:
fig = px.histogram(df2, x="installment",  color_discrete_sequence=['indianred'], nbins=10, title="Installments owed by the borrower")
fig.update_layout(height=800)
fig.show()

As we can see the majority of the borrowers gave 100 to 200 usd as an installement, followed by 200 to 300 usd and very few people gave 700 to 800 usd as an installment




### GRADE AND SUBGRADE

Analyzing the grade and the subgrade assigned by the bank

In [ ]:
grade_df = df2['grade'].value_counts().to_frame()

In [ ]:
grade_df.reset_index(inplace=True)


In [ ]:
grade_df.columns = ['grade', 'count']

In [ ]:

fig = px.bar(grade_df, x="count", y = "grade",  color_discrete_sequence=['teal'])
fig.update_layout(title="Count of the  grade assigned to the Loan applications")
fig.show()

Loan grades are set based on both borrower's credit profile and the nature of the contract. 'A' grade loan represent the lowest risk while G represents the highest risk. 

The borrower's credit profile are set based on these metrics: 

        1. Financial Sale:  Assessment of the company's size by turnover.

        2. Liquidity: Assesment of company's liquidity position, based on last available historical financial statements.

        3. leverage: Assessment of the company's leverage positiion using Net Debt/ Equity ratio

        4. Operation Performance
        
        5.Management Expertise

From the graph we can see that the bank has given out loans that are least riskiest more and this count keeps decreasing with the increasing risk.

Let's analyze if the same trend applies for subgrade

In [ ]:
sub_grade_df = df2['sub_grade'].value_counts().to_frame()
sub_grade_df.reset_index(inplace=True)
sub_grade_df.columns = ['sub_grade', 'count']
fig = px.bar(sub_grade_df, x="count", y = "sub_grade",  color_discrete_sequence=['rosybrown'], text_auto=True)
fig.update_layout(width=800, height=1000, title="Count of the  sub grade assigned to the Loan applications")
fig.show()

The subcategory A4 is the most used in LC.  We can see that the A and B category are used a lot and are in a very close to eachother in number due to the low priority risk. 
If we notice rest of the categories for example C, E, F and G, we see that the subcategory 1 is used a lot more than 2 and 3 and so on. 
With this pattern we can say that 1 is the least riskier than maybe 2 and 3. 

Now sometimes it is possible that people apply for a loan and it is deemed a little more riskier than the other.

For example, a loan to pay the medical bills and loan to buy a car. 

Lot of people who do not have the credit will apply for loan since it is more urgent than say a car since car could be still considered as a luxury in a lot of places.


### HOME OWNERSHIP

In [ ]:
home_ownership_df = df2['home_ownership'].value_counts().to_frame()
home_ownership_df.reset_index(inplace=True)
home_ownership_df.columns = ['home_ownership', 'count']
fig = px.pie(home_ownership_df, values="count", names = "home_ownership", color_discrete_sequence=px.colors.sequential.RdBu)
fig.update_layout(title="Home Ownership status of the applicants")
fig.show()

Majority of the borrower's whose loans were approved were living in rented appartment. 

The next highest clientele are the people who have put their houses on mortgage followed by people who own homes.

People who own homes usually signifies a sign of stability and financial wealth which means they would not need as many loans as the rest but this also makes them the safest people to approve the loans too.

We have a small percentage of clientele in the other section

### Annual Income

Understanding the column using statistical methods

In [ ]:
df2['annual_inc'].describe()

Using a histogram to plot

In [ ]:
fig = px.histogram(df2, x="annual_inc",  color_discrete_sequence=['indianred'], nbins=20, title="Annual Income of the loan applicants")
fig.update_layout(height=800)
fig.show()

As we can see that majority of the applicants are with in the 40k to 50k bracket. 

There are very few applicants that are above 100k and marginally low people who earn in the range of 0 to 10k

The people who apply for loans are those who cannot afford huge amounts right at the moment but have the potential to pay at certain point in the future ( in installements)./

As the annual income increases the number of the loan applicant decreases in step wise nature

If you check the trend for less than 30k, there are not many applicants probably because the bank rejected their applicants as they were not suitable candidates or the customers knew the couldnt pay the loan off.


### verification_status

In [ ]:
df2['verification_status'].unique()

In [ ]:
verification_status_df = df2['verification_status'].value_counts().to_frame()
verification_status_df.reset_index(inplace=True)
verification_status_df.columns = ['verification_status', 'count']
fig = px.pie(verification_status_df, values='count', names='verification_status', color_discrete_sequence=px.colors.sequential.YlGnBu_r)
fig.update_layout(title="Verification status of the applicants")
fig.show()

As we can see that a lot of the applicants are not verified which is not very good for the banks because they could turn out as defaulter. 22% is verified by the source and 27.8 is verified by the bank

### Loan Status

In [ ]:
loan_df = df2['loan_status'].value_counts().to_frame()
loan_df.reset_index(inplace=True)
loan_df.columns = ['loan_status', "Count"]


In [ ]:
fig = px.bar(loan_df, x='loan_status', y = 'Count',  color_discrete_sequence=['pink'])
fig.show()

As we can see that huge percent of the applicants have paid off their loans and and around 500 off them are charged off/ defaulters

### ZIP CODE

In [ ]:
zip_code = df2['zip_code'].value_counts().to_frame()
zip_code.reset_index(inplace=True)
zip_code.columns = ['zip_code', 'count']

In [ ]:
fig = px.bar(zip_code, x='zip_code', y = 'count',  color_discrete_sequence=['darkblue'])
fig.show()

We can see the distribution of the zip code. 

Zip code starting with 100 are highest at the value of 248 followed by zip code in 112



### State of the applicant

In [ ]:
state_df = df2['addr_state'].value_counts().to_frame()
state_df.reset_index(inplace=True)
state_df.columns = ['addr_state', 'count']
fig = px.bar(state_df, x='addr_state', y = 'count',  color_discrete_sequence=['darkred'])
fig.show()

As we can see that majority of the applicants come from california which is one of the most expensive place to live in followed by New York.

The place where least amount of loans were applied were from Mississippi where cost of living is low as well the income is the lowest


### Month where the borrower's earliest report credit line was opened

In [ ]:
earliest_credit_line = df2['earliest_cr_line_month'].value_counts().to_frame().reset_index()
earliest_credit_line.columns = ['Month', 'Count']

In [ ]:
fig = px.bar(earliest_credit_line, x='Month', y = 'Count',  color_discrete_sequence=['purple'])
fig.show()

Majority of the credit lines was opened in October followed by Nov and Dec
which means majority of them are opened in the final quarter of the year probably because of the holiday seasons

## SEGMENTED UNIVARIATE ANALYSIS

### pub_rec
Number of public derogatory records

In [ ]:
df2['pub_rec'].unique()

In [ ]:
pub_rec_df = df2.groupby(['loan_status', 'pub_rec']).size()
pub_rec_df = pub_rec_df.reset_index()
pub_rec_df.columns = ['loan_status', 'pub_rec', 'counts']
pub_rec_df

In [ ]:
px.histogram(data_frame = pub_rec_df
             ,x = 'pub_rec', y = 'counts'
             ,color = 'loan_status')

Majority of the bank loans are given to people with 0  derogatory public records despite that a few people have defaulted.
People with 1 derogatory public records have been given loans who have defaulted. 


### pub_rec_bankruptcies

In [ ]:
df2['pub_rec_bankruptcies'].unique()

In [ ]:
pub_rec_b_df = df2.groupby(['loan_status', 'pub_rec_bankruptcies']).size()
pub_rec_b_df = pub_rec_b_df.reset_index()
pub_rec_b_df.columns = ['loan_status', 'pub_rec_bankruptcies', 'counts']
pub_rec_b_df

In [ ]:
total = []
for r in pub_rec_b_df['pub_rec_bankruptcies'].unique():
   t = pub_rec_b_df[pub_rec_b_df['pub_rec_bankruptcies']== r]['counts'].sum()
   total.append(t)

In [ ]:
total = total * 2

In [ ]:
pub_rec_b_df['counts'] = pub_rec_b_df['counts']/ total

In [ ]:
pub_rec_b_df

In [ ]:
px.histogram(data_frame = pub_rec_b_df
             ,x = 'pub_rec_bankruptcies', y = 'counts'
             ,color = 'loan_status')

More the bankruptcies more the possbility of defaulting

### Loan Amount

Understanding the loan amount requested by people who defaulted/ charged off and fully paid

In [ ]:
px.histogram(data_frame = df2
             ,x = 'loan_amnt'
             ,color = 'loan_status', marginal="box",
             )

If you see distribution plot and the violin plot the loan amount is slightly higher for the defaulter than those who have fully paid off. The median of the full paid customers is 7200 and for defaulter it is 9000 usd. Loans applied for by the full paid people are on the lower end than those who defaulted. 

### Employee Length

In [ ]:
df_emp_length = df2.groupby(by=["emp_length", "loan_status"]).size().reset_index(name="counts")
px.bar(data_frame=df_emp_length, x="emp_length", y="counts", color="loan_status", barmode="group")

In [ ]:

count_emp = df2.groupby(by=["emp_length"]).size().reset_index(name="counts")
final_emp_df = pd.merge(df_emp_length, count_emp, on='emp_length', how="inner")
final_emp_df.columns = ['emp_length', 'loan_status', 'counts', 'total']


In [ ]:
final_emp_df['counts'] = final_emp_df['counts']/ final_emp_df['total'] * 100

In [ ]:
px.bar(data_frame=final_emp_df, x="emp_length", y="counts", color="loan_status", barmode="group")

In [ ]:
final_emp_df[final_emp_df['loan_status'] == "Charged Off"].sort_values("counts")

1. People who are most likely to get the loan have more than 10 or 10+ years experience

2. The maximum defaulters are in work experience 10 or 10 above bracket




### Term

In [ ]:
df_term = df2.groupby(by=["term", "loan_status"]).size().reset_index(name="counts")

In [ ]:
px.bar(data_frame=df_term, x="term", y="counts", color="loan_status", barmode="group")

The people who have fully paid off their loans prefer the 36 months term compared to 60 months but for the defaulters we can see that prefer 60 months time periods over 36 months

### Interest Rate

In [ ]:
hist_data = [df2[df2["loan_status"] == "Charged Off"]['int_rate'].values, df2[df2["loan_status"] == "Fully Paid"]['int_rate'].values]

group_labels = ['Charged Off', 'Fully Paid']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
fig.show()

This is a distribution plot for interest rate and these are the following conclusions:

1. The frequency of the rate of interest is high for people who have paid off their loans ie people who paid off their loans have cheaper rate of interest
2. The interests almost intesects at 9% for both the categories and then it is overtaken for the charged off people after 11 %



### Installements

In [ ]:
fig = px.box(df2, x="loan_status", y="installment", points="all")
fig.show()

The median installment to be paid by defaulters and those who have fully paid the loan is almost similar. The installment is slight on the higher end for the charged off customers.

### Grade and Subgrades

In [ ]:
df_grade = df2.groupby(by=["grade", "loan_status"]).size().reset_index(name="counts")
px.bar(data_frame=df_grade, x="grade", y="counts", color="loan_status", barmode="group")

Inferences:

1. For fully paid customers, the highest number of applications are for A type loan and it reduces in a fashionable manner - which means that there are few customers whose loans are risky. A being the least riskiest.

2. For Charged off customers, Grade Type B seems to be the highest after which the number of applications per risk type keep reducing probably because those loans that looked too risky were not approved by the bank to begin with.

###  Home ownership

In [ ]:
df_home = df2.groupby(by=["home_ownership", "loan_status"]).size().reset_index(name="counts")
px.bar(data_frame=df_home, y="home_ownership", x="counts", color="loan_status", barmode="group")

For Full paid customers
Majority of the people live in rent homes followed by mortgaged homes and with few people living in their own homes. This could be the case probably because they invest their money in some other forms like stocks , rather than invest in real estate

For Charged off customers: The home ownership status is the same

### Annual Income

In [ ]:
fig = px.box(df2, x="loan_status", y="annual_inc", points="all", color_discrete_sequence=['purple'])
fig.show()

From the above plot we can see that annual income of the people who have fully paid off their loans is on the higher end comparared to the people who have been charged off.

### Verification Status

In [ ]:
verification_status_grouped = pd.pivot_table(df2, index=['loan_status', 'verification_status'], values='loan_amnt', aggfunc=len)
verification_status_grouped.columns = ['count']

In [ ]:

verification_status_grouped.reset_index(inplace=True)

In [ ]:
verification_status_grouped

In [ ]:
fig = px.bar(verification_status_grouped, x="loan_status", y="count",
             color='verification_status', barmode='group',
             height=400)
fig.update_layout(title="Comparision of verification status with respect to loan_status")
fig.show()

We can see that lot of the loan applications are not verified for both fully paid and charged off which is a practice that could be changed. 

For fully paid loans the source verified is more than verified which is opposite for charged off.

### 

### Checking the zip code to see where the people who fully paid their loans and defaulters come from

In [ ]:
zip_code_grouped = df2.groupby(['zip_code', 'loan_status'])['loan_amnt'].count()
zip_code_grouped.columns = ['Count']

In [ ]:
zip_code_grouped.head(20)

Since there are so many zip codes lets just go with the first 20 records
Not every zip code has both the categories charged off and fully paid

### State of each of the types

In [ ]:
df_home = df2.groupby(by=["addr_state", "loan_status"]).size().reset_index(name="counts")
fig = px.bar(data_frame=df_home, y="addr_state", x="counts", color="loan_status", barmode="group", color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_layout(height=1000)
fig.show()


Majority of the loan applicants are from the californian region - both defaulted as well as full paid

There are few states in which there are no defaulters at all probably because the loan was never sanctioned to them. 

The defaulters are mostly in the developed cities. Probably who are middle class and lower middle class in those cities


### Inquiry in last 6 months

In [ ]:
inquiry_grouped = df2.groupby(['loan_status'])['inq_last_6mths'].sum()
inquiry_grouped = inquiry_grouped.to_frame()
inquiry_grouped = inquiry_grouped.reset_index()

In [ ]:
px.bar(inquiry_grouped, x="loan_status", y="inq_last_6mths")

This graph is biased since the number of records for charged off applicants are low so we will do a ratio of inquries to the number of applicants

In [ ]:

inquiry_grouped.at[0, "inq_last_6mths"] = float(inquiry_grouped[inquiry_grouped['loan_status'] == "Charged Off"]['inq_last_6mths'] / len(df2[df2['loan_status'] == "Charged Off"]))

inquiry_grouped.at[1, "inq_last_6mths"] = float(inquiry_grouped[inquiry_grouped['loan_status'] == "Fully Paid"]['inq_last_6mths'] / len(df2[df2['loan_status'] == "Fully Paid"]))


In [ ]:
px.bar(inquiry_grouped, x="loan_status", y="inq_last_6mths")

When we check the ratio we see that the charged off people have had more inquries than people who paid off their loans


### Open Credit Lines

Checking the credit lines per customer

In [ ]:
credit_line_grouped = df2.groupby(['loan_status'])['open_acc'].sum()
credit_line_grouped = credit_line_grouped.to_frame()
credit_line_grouped = credit_line_grouped.reset_index()

credit_line_grouped.at[0, "open_acc"] = float(credit_line_grouped[credit_line_grouped['loan_status'] == "Charged Off"]['open_acc'] / len(df2[df2['loan_status'] == "Charged Off"]))

credit_line_grouped.at[1, "open_acc"] = float(credit_line_grouped[credit_line_grouped['loan_status'] == "Fully Paid"]['open_acc'] / len(df2[df2['loan_status'] == "Fully Paid"]))
px.bar(credit_line_grouped, x="loan_status", y="open_acc")

Credit Line per customer is the comparable for both categories

### Total Account

In [ ]:
total_account_line_grouped = df2.groupby(['loan_status'])['total_acc'].sum()
total_account_grouped = total_account_line_grouped.to_frame()
total_account_grouped = total_account_grouped.reset_index()

total_account_grouped.at[0, "total_acc"] = float(total_account_grouped[total_account_grouped['loan_status'] == "Charged Off"]['total_acc'] / len(df2[df2['loan_status'] == "Charged Off"]))

total_account_grouped.at[1, "total_acc"] = float(total_account_grouped[total_account_grouped['loan_status'] == "Fully Paid"]['total_acc'] / len(df2[df2['loan_status'] == "Fully Paid"]))
px.bar(total_account_grouped, x="loan_status", y="total_acc", color_discrete_sequence=['purple'])

### Total Received Principal and Total Received Interest


In [ ]:
total_prin_inr_grouped = df2.groupby(['loan_status'])['total_rec_prncp', 'total_rec_int'].mean()
total_prin_inr_grouped = total_prin_inr_grouped.reset_index()

In [ ]:
px.bar(total_prin_inr_grouped, x="loan_status", y=["total_rec_prncp", "total_rec_int"])

In this graph we are comparing two column wrt to total_rec_prncp and total_rec_int

Principal amount for the fully paid is higher than charged off by a significant amount but when you compare the interest rate they are very comparable. 



## BIVARIATE ANALYSIS

### Understanding the corelation between numeric variables

In [ ]:
numeric_cols = df2._get_numeric_data()
corr = numeric_cols.corr()
sns.set(rc={"figure.figsize":(40, 20)})
cmap = sns.diverging_palette(230, 20, as_cmap=True)
fig = sns.heatmap(corr, annot=True,cmap=cmap)

Inferences from the above plot:
1. Loan amount has high correlation with total_payment, total_payment_inv and total_rec_prncp, funded_amount and funded_amt_inv.

2. Funded amount has high correlation with total_payment, total_payemnt_inv, total_rec_prncp, loan_amt and funded_amt_inv


2. Installments has high correlation with total_payment, total_payment_inv and total_rec_prncp, funded_amt and funded_amt_inv


### Understanding the relation between the loan amount and total installments

In [ ]:
fig = px.scatter(df2, x="loan_amnt", y="installment", color="loan_status")
fig.show()


The pattern followed by both the loan status are very similar except for some points where despite the loan amount being high the installement given is very low and yet they manage to pay off the loan.

The installement paid by the defaulters is relatively on the higher side compared to the people who have paid off their loan probably because they were paying off the default money


### Understanding the term and the interest rate relation for different types of customer

In [ ]:
fig = px.violin(df2, x="term", y="int_rate", color="loan_status")
fig.show()

For customers in 36 months term loan, we can see that the interest rate for the majority fully paid is in the range of 5 to 10 where as for defaulters it is in the range of 10 to 15

For customers in the term of 60 momths the interest rate for fully paid off people is higher compared to 36 months and is slightly lower to defaulters (majority) in the 60 months category

### Understanding the relationship between home ownership and the rate of interest

In [ ]:
fig = px.box(df2, x="home_ownership", y="int_rate", color="loan_status")
fig.show()

For fully paid off people, the interest rate is higher for Other category followed by rent, own and mortgage being the lowest

For charged off customers, the interest rate is higher in their category but it is the lowest for other, people who own the houses followed by mortgage and then rent. 

### Understanding the relationship between interest rate and inquiry

In [ ]:
fig = px.histogram(df2, x="int_rate", y="inq_last_6mths", color="loan_status")
fig.show()

### Understanding the relationship between loan amount and grade

In [ ]:
fig = px.violin(df2, x="grade", y="loan_amnt", color="loan_status", color_discrete_sequence=px.colors.sequential.Plasma)
fig.show()

For charged off loans, Grade C has the maximum range of the loan amount . In grade F , majority of the loan applicants in the upper range of the violin plot unlike G which is distributed through out. 

For fully paid loans: the max range goes to 28k but it is still categorized in A. For the same value for charged off person, it is present in Category C. 

### Understanding the relationship between home ownership and loan amount

In [ ]:
owner_loan=pd.pivot_table(df2, index=['home_ownership'], values=['loan_amnt'], aggfunc='mean')
owner_loan = owner_loan.reset_index()

In [ ]:
px.bar(owner_loan, x="home_ownership", y="loan_amnt", color_discrete_sequence=['orange'])

People who either had a mortgage or were in other category had higher loans in compared to people with own homes and rent.

People with own homes have more stability and are financial well off hence the loans could be lower. 

## INFERENCES

Plot wise inferences are mentioned along with the visualization.
Following are the metrics the bank should look out for to identify a defaulter:
1. Loan Amount: People who default have a relatively higher loan amount compared to those who fully pay off their loans
2. Employee Length: The bank approves loan easily for the people who have 10 or more than 10 years of experience but they also tend to be huge defaulters. The next highest number of defaulters are the people with less than 1 years of experience.
3. Term: People who default tend to take loans on 60 months than 36 months
4. Interest rate: People with higher rate of interest are more likely to be defaulters.
5. Grade: For fully paid customers, the highest number of applications are for A type loan and it reduces in a fashionable manner - which means that there are few customers whose loans are risky. A being the least riskiest.For Charged off customers, Grade Type B seems to be the highest after which the number of applications per risk type keep reducing probably because those loans that looked too risky were not approved by the bank to begin with.
6. Home ownership: People who own houses have more stability and the defaulters in this case are lower.
7. Inquiry in Last 6 months: People who have had large number of inquries are more likely to default
8. Open Credit Lines: Charged off people tend to have more open credit lines. 
